# Model Comparison

## Context

With our LightGBM properly trained, we can move on to constructing the hybrid model, and we have several possible approaches. One option is simple boosting, in other words, subtract the LightGBM predictions from our Dynamic Harmonic Regression (DHR) forecast. Another option is to retrain the DHR while including the LightGBM predictions as an exogenous variable. A third alternative is to use the LightGBM predictions as the forecasted error term in the MA component. All three approaches are valid and will be implemented for comparison against each other and against the baseline: the standalone DHR model.

**Data Source**
The data used in this notebook was extracted from the notebook *exploratory_analysis/eda-linear.ipynb*

- **Data:** 08/12/2025
- **Localização:** ../data/wrangle

## Set up

### Libraries

In [130]:
## Base
import os
import pickle
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

## Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Model
import statsmodels.tsa.arima_model as test

In [8]:
# Funções criadas
import sys
from pathlib import Path
sys.path.insert(1, Path.cwd().parents[1].as_posix())

from src.ts_utils import *

from config import *

In [9]:
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#003366'])

In [10]:
def fourier(order, period, t):
    comp = pd.DataFrame(index=range(0, t))
    for k in range(1, order+1):
        comp[f'sin_{k}'] = np.sin(2*np.pi*k*comp.index/period)
        comp[f'cos_{k}'] = np.cos(2*np.pi*k*comp.index/period)
        
    return comp

# Data

For this notebook, we will use all available data. Before training the models, we must engineer the necessary features.

In [11]:
df = pd.read_parquet(os.path.join(DATA_PATH_WRANGLE, 'weather_sanitized.parquet'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007 entries, 0 to 4006
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    4007 non-null   datetime64[ns]
 1   tavg    4007 non-null   float64       
 2   prcp    4007 non-null   float64       
 3   snow    4007 non-null   float64       
 4   wspd    4007 non-null   float64       
 5   pres    4007 non-null   float64       
 6   tamp    4007 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 219.3 KB


LightGBM features

In [12]:
light_feature = ['pres_703', 'tamp_404', 'prcp_702', 'tamp_709_diff', 'wspd_718', 'prcp_411', 'wspd_646', 'tamp_582', 'tamp_523_diff', 'tamp_372', 'prcp_382', 'tamp_469', 
                 'prcp_703', 'tavg_378_diff', 'wspd_657', 'tavg_412', 'wspd_416', 'tavg_486_diff', 'wspd_524', 'tamp_397_diff', 'snow_370_diff', 'pres_389_diff', 'wspd_597', 
                 'pres_440_diff', 'tavg_682_diff', 'snow_694', 'tavg_695_diff', 'tamp_532_diff', 'tamp_382', 'wspd_401_diff', 'tavg_728', 'tamp_691_diff', 'prcp_376', 
                 'tamp_403', 'tamp_490', 'tamp_375', 'wspd_464_diff', 'tamp_720_diff']

In [13]:
for feat in light_feature:
    if '_diff' in feat:
        origin, lag, _ = feat.split('_')
        df[feat] = df[origin].shift(int(lag)).diff()
    else:
        origin, lag = feat.split('_')
        df[feat] = df[origin].shift(int(lag))

Fourier Terms

In [14]:
df = pd.concat([df, fourier(1, 365, df.shape[0])], axis=1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007 entries, 0 to 4006
Data columns (total 47 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   time           4007 non-null   datetime64[ns]
 1   tavg           4007 non-null   float64       
 2   prcp           4007 non-null   float64       
 3   snow           4007 non-null   float64       
 4   wspd           4007 non-null   float64       
 5   pres           4007 non-null   float64       
 6   tamp           4007 non-null   float64       
 7   pres_703       3304 non-null   float64       
 8   tamp_404       3603 non-null   float64       
 9   prcp_702       3305 non-null   float64       
 10  tamp_709_diff  3297 non-null   float64       
 11  wspd_718       3289 non-null   float64       
 12  prcp_411       3596 non-null   float64       
 13  wspd_646       3361 non-null   float64       
 14  tamp_582       3425 non-null   float64       
 15  tamp_523_diff  3483 n

In [18]:
df_train = df.head(7 * 365).copy()
df_test = df.head(9 * 365).tail(2 * 365).copy()

# Models Training

## Dynamic Harmonic Regression

In [131]:
DHR = test.ARIMA(df_train['tavg'].array, df_train[["sin_1", "cos_1"]], order=(2,0,2)).fit()

NotImplementedError: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been removed in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and statsmodels.tsa.SARIMAX.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained. It also offers alternative specialized
parameter estimators.


In [38]:
DHR.predict(h=1, X=df_test[["sin_1", "cos_1"]].head(1).to_numpy())

{'mean': array([272.53466422])}

In [41]:
DHR.

'CSS-ML'

In [23]:
sin_1, cos_1, ar1, ar2, ma1, ma2, _ = DHR.params

In [24]:
(sin_1 * df_test["sin_1"].tolist()[0]) + (cos_1 * df_test["cos_1"].tolist()[0]) +\
        (ar1 * df_train["tavg"].tolist()[-1]) + (ar2 * df_train["tavg"].tolist()[-2]) +\
        (ma1 * DHR.resid.tolist()[-1]) + (ma2 * DHR.resid.tolist()[-2])

266.43481971918385

In [26]:
DHR.forecast(1, exog=df_test[["sin_1", "cos_1"]].head(1))

2555    272.798869
dtype: float64

In [ ]:
fr = DHR.filter_results
a_pred = fr.predicted_state[:, -1]         # last predicted state
Z = fr.model.design[:, :, -1]              # observation matrix
exog_contrib = DHR.params()['intercept'] + \
               DHR.params()['sin_1']*df_test['sin_1'].iloc[0] + \
               DHR.params()['cos_1']*df_test['cos_1'].iloc[0]
yhat = float(Z @ a_pred) + exog_contrib


In [ ]:
DHR.forecast(1, exog=df_test[["sin_1", "cos_1"]].head(1))

In [ ]:
DHR.params

## Boosting
In the boosting version, we simply subtract the DHR forecast to the LightGBM prediction. Therefore, we only need to save the predictions in our test set for later comparison.

In [ ]:
with open('lightgbm_model.pkl', 'rb') as f:
    tree_model = pickle.load(f)
tree_model

In [ ]:
df_test['light_pred'] = tree_model.predict(df_test[light_feature])

## Dynamic Harmonic Regression + XLight
For this model, we retrain our DHR using the LightGBM predictions for the training data as an exogenous variable. Two important notes: since LightGBM was trained on this data, the hybrid model may unintentionally become overfitted, and because the LightGBM features are lagged, the DHR will have fewer observations available for training.

In [ ]:
df_train = df_train.dropna().reset_index(drop=True)
df_train['light_pred'] = tree_model.predict(df_train[light_feature])

In [ ]:
DHRXL = ARIMA(df_train['tavg'], df_train[["sin_1", "cos_1", "light_pred"]], order=(2,0,2)).fit()
DHRXL.summary()

## Dynamic Harmonic Regression & XLight

For this final model, we will manually construct the predictions using the tavg and light_pred features, along with the DHR residuals and forecasts, combined with the coefficients from our original DHR model.

# Expanding cross-validation

In this final section, we use expanding cross-validation on two years of data to evaluate the performance of each model. Our comparison metric is Mean Absolute Error (MAE).

In [ ]:
df_ecv = pd.concat([df_train, df_test], ignore_index=True).sort_values('time')
X = df_ecv[["sin_1", "cos_1", "light_pred"]].copy()
y = df_ecv['tavg'].copy()

In [ ]:
error = {'DHR': [], 'Boost': [], 'DHRXL': []}

for i in tqdm(range(-2*365, -727), position=0, leave=True):
    # Sets
    X_train = X[:i]
    X_valid = X[i:i+365]
    y_train = y[:i]
    y_valid = y[i:i+365]
    
    # DHR model
    ## Train Model
    DHR = ARIMA(y_train, X_train[["sin_1", "cos_1"]], order=(2,0,2)).fit()
    
    ## Forecast
    DHR_forecast = DHR.forecast(365, exog=X_valid[["sin_1", "cos_1"]])
    error['DHR'].append(abs(DHR_forecast - y_valid).reset_index(drop=True))

    # Boosting model
    Boost_forecast = DHR_forecast - X_valid['light_pred']
    error['Boost'].append(abs(Boost_forecast - y_valid).reset_index(drop=True))

    # DHRXL
    ## Train Model
    DHRXL = ARIMA(df_train['tavg'], df_train[["sin_1", "cos_1", "light_pred"]], order=(2,0,2)).fit()

    ## Forecast
    DHRXL_forecast = DHRXL.forecast(365, exog=X_valid[["sin_1", "cos_1", "light_pred"]])
    error['DHRXL'].append(abs(DHRXL_forecast - y_valid).reset_index(drop=True))

    # DHR_light_errors
    const, sin_1, cos_1, ar1, ar2, ma1, ma2, _ = DHR.params


In [ ]:
sin[1]

In [ ]:
y = y_train[-2:].tolist()
e = DHR.resid[-2:].tolist()
sin = X_valid['sin_1'].tolist()
cos = X_valid['cos_1'].tolist()

i=1
pred = const + (sin_1 * sin[i]) + (cos_1 * cos[i]) + (ar1 * y[-1]) + (ar2 * y[-2]) + (ma1 * e[-1]) + (ma2 * e[-2])

In [ ]:
pred

In [ ]:
DHR.resid

In [ ]:
DHR_forecast

In [ ]:
dhr_mae = pd.concat(error['DHR'], axis=1, ignore_index=True).mean(axis=1)
boost_mae = pd.concat(error['Boost'], axis=1, ignore_index=True).mean(axis=1)
dhrxl_mae = pd.concat(error['DHRXL'], axis=1, ignore_index=True).mean(axis=1)

In [ ]:
dhrxl_mae.describe()

In [ ]:
boost_mae.describe() # -

In [ ]:
const, sin_1, cos_1, ar1, ar2, ma1, ma2, _ = DHR.params